In [2]:
import keras

/Users/DTH/.local/share/virtualenvs/CDA_Case2-Q7vfWjH5/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
!pip list | grep Keras

Keras            2.1.6      


In [12]:
import numpy as np
import pandas as pd
import glob
from scipy import misc

from HandleDMIFeatures import DMI_Handling as HDMI

import numpy as np
import os
import mahotas as mh
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics, model_selection
from scipy.stats import norm

import matplotlib.pyplot as plt

import keras
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.applications.vgg16 import preprocess_input

# 1 Load data

## 1.1 Parse data

In [13]:
#top_folder = "..\\data\\online_rep\\data\\"
top_folder = "online_rep/"

foggy_im_path = top_folder + "foggy/"
clear_im_path = top_folder + "clear/"
net_im_path = top_folder + "net/"

foggy_im = HDMI.get_im_files(foggy_im_path)
clear_im = HDMI.get_im_files(clear_im_path)
net_im = HDMI.get_im_files(net_im_path)

all_images = [foggy_im, clear_im]


# Flatten, so one list has all paths
all_images_flat = [item for sublist in all_images for item in sublist]

In [14]:
# Build X and y

## X
dim = (224, 224, 3)
N = len(all_images_flat)
X = np.zeros((N, dim[0], dim[1], dim[2]))
for i, img_path in enumerate(all_images_flat):
    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    X[i, :, :, :] = preprocess_input(img)
    
#X = X.astype(np.uint8)

## y
y_listed = np.zeros(len(all_images_flat))
run_index = 0
for i, images in enumerate(all_images):
    if i == 0:
        # First response is 0
        run_index = len(images)
    else:
        # Speghetti code
        y_listed[run_index:(run_index + len(images))] = i
        run_index += len(images)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y_listed, shuffle=True)

# 2 Modelling

In [16]:
conv = VGG16(weights='imagenet', include_top=False)

# create new input layer
input_layer = keras.layers.Input(shape=X_train.shape[1:], name="image_input")

# output from feature extraction of input
features = conv(input_layer)

# fully connected layers
x = keras.layers.Flatten(name="flatten")(features)
x = keras.layers.Dense(3, activation="relu", name="fully_connected1")(x)
#x = keras.layers.Dropout(0.50)(x)
x = keras.layers.Dense(1, activation="sigmoid", name="yhat")(x)


# create model
model = keras.models.Model(inputs=input_layer, outputs=x)

# freeze VGG16 conv layers
for layer in conv.layers:
    layer.trainable = False

# optimizer
#optimizer = keras.optimizers.adam(lr=0.0001)
    
# compile model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
fully_connected1 (Dense)     (None, 3)                 75267     
_________________________________________________________________
yhat (Dense)                 (None, 1)                 4         
Total params: 14,789,959
Trainable params: 75,271
Non-trainable params: 14,714,688
_________________________________________________________________


## 2.1 Train - Test split

In [18]:
results = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=17)

Train on 0 samples, validate on 0 samples
Epoch 1/10


AttributeError: 'ProgbarLogger' object has no attribute 'log_values'

## 2.1 Cross-validation

# 3 Evaluation